In [4]:
import geopandas as gpd
import pandas as pd
import numpy as np
import random
import sys, os.path, os
from tqdm import tqdm
import pickle as pkl
from shapely.geometry import Point,Polygon
import dateutil.parser

import random
from datetime import datetime, timedelta

import sys, os.path, os
try:
    from google.colab import drive
    drive.mount('/content/drive')
    path_opencp="drive/Shareddrives/FAIRNESS/Colab/Librerias/PredictCode/"
    path_fairness="drive/Shareddrives/FAIRNESS/Colab/Librerias/"
    path_exp="drive/Shareddrives/FAIRNESS/Colab/Notebooks_for_topic/EXP0/SCRIPTS/"

except:

    path_opencp="g:\\Unidades compartidas\\FAIRNESS\\Colab\\Librerias\\PredictCode"
    path_fairness="g:\\Unidades compartidas\\FAIRNESS\\Colab\\Librerias"
    path_exp="g:\\Unidades compartidas\\FAIRNESS\\Colab\\Notebooks_for_topic\\EXP0\\SCRIPTS"

sys.path.insert(0, os.path.abspath(path_opencp))
sys.path.insert(0, os.path.abspath(path_fairness))
sys.path.insert(0, os.path.abspath(path_exp))

import open_cp
import open_cp.geometry
import matplotlib.pyplot as plt

try:
    os.chdir("drive/Shareddrives/FAIRNESS/Colab/Notebooks_for_topic/EXP0/DATOS/BOGOTA")
except:
    os.chdir("g:\\Unidades compartidas\\FAIRNESS\\Colab\\Notebooks_for_topic\\EXP0\\DATOS\\BOGOTA")


Mounted at /content/drive


ERROR:open_cp.geometry:Failed to import `rtree`.


In [5]:
from models.model_selection import NAIVE_MODEL, KDE_MODEL, SEPP_MODEL
from open_cp.sources.sepp import make_time_unit
from global_vars import days_time_unit
import datetime
from datetime import timedelta

def intensity_m(model,date,region,grid_size):

    try:
        predict=model.predict(date)
    except:
        predict=model.predict()

    try:
        M=open_cp.predictors.GridPredictionArray.from_continuous_prediction_region(predict, region, grid_size, grid_size)
        intensity_matrix= M.intensity_matrix
    except:
        intensity_matrix = predict.intensity_matrix

    if intensity_matrix.sum()==0:
        return intensity_matrix

    else:
        return intensity_matrix/intensity_matrix.sum()

### datos reales
import open_cp.naive as naive
def get_real_m(timedpoints,p_date,grid_size,region):
    try:
        real = timedpoints[(timedpoints.times_datetime()>=p_date) & ((timedpoints.times_datetime()<p_date+timedelta(days=1)))]
    except:
        p_date=datetime.datetime.combine(pred_date,datetime.time())
        real = timedpoints[(timedpoints.times_datetime()>=p_date) & ((timedpoints.times_datetime()<p_date+timedelta(days=1)))]

    predictor = naive.CountingGridKernel(grid_size,grid_size,region=region)
    predictor.data = real
    try:
        gridpred = predictor.predict().renormalise()
        real=gridpred.intensity_matrix.data

        return real
    except:
        return predictor.predict().intensity_matrix

Failed to load 'descartes' package.
ERROR:open_cp.network:Failed to import `rtree`.


## <center> <font color='red'>7. Train and predict Models

In [6]:
Train_bog = pkl.load(open("Train_bog.pkl", "rb")) ## cargar datos
Test_bog = pkl.load(open("Test_bog.pkl", "rb")) ## cargar datos
region = pkl.load(open("definition_region.pkl", "rb")) ## cargar datos

In [7]:
import open_cp.seppexp as sepp

In [8]:
### SEPP
path_file=os.path.join("SEPP/modelgrid.pkl")

if not os.path.exists(path_file):
    trainer = sepp.SEPPTrainer(region, grid_size=1000)
    trainer.data = Train_bog
    sepp_model_result = trainer.train(iterations=50)
    sepp_model_result.data = Train_bog
    pkl.dump(sepp_model_result, open(path_file, "wb"))

else:
    sepp_model_result=pkl.load(open(path_file, "rb"))

/content/drive/Shareddrives/FAIRNESS/Colab/Librerias/PredictCode/open_cp/seppexp.py:191: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  cells = _np.empty((ysize, xsize), dtype=_np.object)


AttributeError: module 'numpy' has no attribute 'object'.
`np.object` was a deprecated alias for the builtin `object`. To avoid this error in existing code, use `object` by itself. Doing this will not modify any behavior and is safe. 
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [ ]:
if not os.path.exists(os.path.join("SEPP/","predictions_grid.pkl")):
    predictions={}
else:
    predictions=pkl.load(open(os.path.join("SEPP/","predictions_grid.pkl"), "rb"))

if not os.path.exists(os.path.join("SEPP/","reals_grid.pkl")):
    reals={}
else:
    reals=pkl.load(open(os.path.join("SEPP/","reals_grid.pkl"), "rb"))


for i in tqdm(range(30)):
    pred_date=Test_bog.time_range[0].astype(datetime.datetime).date()+timedelta(days=i)
    if not pred_date in predictions:
        predict=intensity_m(sepp_model_result,pred_date,region,1000)
        predictions[pred_date]=predict

    # real data
    if not pred_date in reals:
        real=get_real_m(Test_bog,pred_date,1000,region)
        reals[pred_date]=real
    # try:
    #     EMD[pred_date]=EMD(real,predict,25000000)
    # except:
    #     pass


    pkl.dump(predictions, open(os.path.join("SEPP/","predictions_grid.pkl"), "wb"))
    pkl.dump(reals, open(os.path.join("SEPP/","reals_grid.pkl"), "wb"))


100%|██████████| 30/30 [00:00<00:00, 35.70it/s]
